In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model

In [2]:
# đường dẫn tới file chứa dữ liệu câu hỏi và câu trả lời
data_path = './chatDataSet.txt'

# đọc dữ liệu từ file
with open(data_path, 'r', encoding='utf8') as f:
    lines = f.read().split('\n')

In [3]:
questions = [line.split(":")[0] for line in lines if len(line.split(':')) == 2]
answers = [line.split(":")[1] for line in lines if len(line.split(':')) == 2]

In [4]:
# Khởi tạo Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)

# Chuyển các câu thành dạng số
question_seqs = tokenizer.texts_to_sequences(questions)
answer_seqs = tokenizer.texts_to_sequences(answers)

# Thêm padding để các câu có cùng độ dài
maxlen = 10
padded_question_seqs = pad_sequences(question_seqs, maxlen=maxlen, padding="post")
padded_answer_seqs = pad_sequences(answer_seqs, maxlen=maxlen, padding="post")

# Xác định số từ và số chiều của embedding
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 200

In [5]:
# mô hình
encoder_inputs = Input(shape=(maxlen,))
encoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(embedding_dim, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(maxlen,))
decoder_embedding = Embedding(vocab_size, embedding_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(embedding_dim, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation="softmax")
output = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], output)

# Compile và huấn luyện mô hình
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 10, 200)      1002600     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 10, 200)      1002600     ['input_2[0][0]']                
                                                                                              

In [25]:
from keras.utils import to_categorical

# Chuyển label sang dạng one-hot encoding
one_hot_answers = np.zeros((len(answers), maxlen, vocab_size), dtype="float32")
for i, answer_seq in enumerate(padded_answer_seqs):
    one_hot_answers[i] = to_categorical(answer_seq, num_classes=vocab_size)
    
# train
batch_size = 32
epochs = 1
with tf.device('/gpu:0'):
    history = model.fit([padded_question_seqs, padded_answer_seqs], one_hot_answers, batch_size=batch_size, epochs=epochs)

402/402 [==============================] - 36s 88ms/step - loss: 0.2490


In [26]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [27]:
encoder_model , decoder_model = make_inference_models()

In [29]:
# Chuẩn bị dữ liệu đầu vào
question = ["hi hi hi hi hi hi"]
question_seq = tokenizer.texts_to_sequences(question)
question_padded = pad_sequences(question_seq, maxlen=maxlen, padding="post")

# Mã hóa câu hỏi và khởi tạo state đầu vào cho decoder
states_value = encoder_model.predict(question_padded)
target_seq = np.zeros((1, 1))

# Tạo câu trả lời
stop_condition = False
decoded_sentence = ''
index_to_word = {index: word for word, index in tokenizer.word_index.items()}
while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    print(sampled_token_index)
    sampled_word = index_to_word[sampled_token_index]
    decoded_sentence += ' ' + sampled_word
    if (sampled_word == '<end>' or len(decoded_sentence) > maxlen):
        stop_condition = True
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = sampled_token_index
    states_value = [h, c]

print(decoded_sentence)

1/1 [==============================] - 0s 26ms/step
295
1/1 [==============================] - 0s 20ms/step
295
1/1 [==============================] - 0s 31ms/step
295
 ngày ngày ngày


In [30]:
index_to_word = {index: word for word, index in tokenizer.word_index.items()}
index_to_word

{1: 'em',
 2: 'chào',
 3: 'mày',
 4: 'anh',
 5: 'con',
 6: 'là',
 7: 'e',
 8: 'ngu',
 9: 'đi',
 10: 'tao',
 11: 'chó',
 12: 'a',
 13: 'chị',
 14: 'mẹ',
 15: 'có',
 16: 'cho',
 17: 'nói',
 18: 'dạ',
 19: 'cc',
 20: 'bạn',
 21: 'không',
 22: 'm',
 23: 'thì',
 24: 'ạ',
 25: 'yêu',
 26: 'ko',
 27: 'gì',
 28: 'kệ',
 29: 'cái',
 30: 'ăn',
 31: 'mà',
 32: 'nó',
 33: 'như',
 34: 'vâng',
 35: 'địt',
 36: 'rồi',
 37: 'bot',
 38: 'xin',
 39: 'ok',
 40: 'lắm',
 41: 't',
 42: 'à',
 43: 'đéo',
 44: 'quá',
 45: 'thằng',
 46: 'sao',
 47: 'làm',
 48: 'sẽ',
 49: 'của',
 50: 'chịch',
 51: 'đẹp',
 52: 'thế',
 53: 'ra',
 54: 'điên',
 55: 'á',
 56: 'tôi',
 57: 'nha',
 58: 'biết',
 59: 'đó',
 60: 'xl',
 61: 'lồn',
 62: 'nhau',
 63: 'lỗi',
 64: 'c',
 65: 'cứt',
 66: 'k',
 67: 'j',
 68: 'vậy',
 69: 'bố',
 70: 'nữa',
 71: 'cũng',
 72: 'thôi',
 73: 'chứ',
 74: 'bị',
 75: 'luôn',
 76: 'ai',
 77: 'lại',
 78: 'vào',
 79: 'xấu',
 80: 'hơn',
 81: 'đâu',
 82: 'thích',
 83: 'người',
 84: 'đây',
 85: 'xinh',
 86: 'mình'